# JDFTx

This tutorial will teach you the basic workflow of computational chemistry using JDFTx as our DFT software. This tutorial won't cover any theory, just application and methods. This tutorial also assumes that you will be doing high throughput electrochemical material discovery, and will teach the concepts needed to perform those experiments.

Take a look at the computational workflow below. This will be generally applicable to anything done in our group.

<img src="../Pictures/JDFTx/Computational_Workflow.png" alt='Computational Workflow'>

The Structure Generation phase will be easy throughout these tutorials, because all of the necessary structure files will be supplied in the files folder. 

The Ionic Relaxation is the majority of the computational time and is highly important. During this phase, the initial structure will be "relaxed," meaning the nucleus positions will be shifted to lower the forces between them and minimize the total energy. These forces are generated by running DFT and solving the many body problem for the electrons and are then used by a minimization algorithm to change nuclear positions. We can set both energetic and force related convergence criteria, which we'll do later. A structure is considered relaxed when it clears the force/energy convergence criteria we set.

Following ionic relaxation, we need to do one more electronic structure calculation that we label the Singlepoint. This calculation won't shift the nuclear positions, just generate electronic structure parameters such as the density of states, spatial orbitals, and energetics.

To understand this workflow, we'll go through a few examples, starting with a nitrogen molecule.

## Nitrogen Molecule

To submit any calculation in JDFTx, we must have a specifically formatted file called the _in_ file. This file will contain the geometry of our molecule as well as the DFT parameters we want to use for the calculation. In our group, we use a file called the __POSCAR__ (pos- as in position) to represent the geometry of our molecule. Let's look at the POSCAR for nitrogen, found in the N2/ directory in the files/ directory.

 ```
 N 
 1.0000000000000000
    10.0000000000000000    0.0000000000000000    0.0000000000000000
     0.0000000000000000   10.0000000000000000    0.0000000000000000
     0.0000000000000000    0.0000000000000000   10.0000000000000000
 N 
   2
Cartesian
  0.0000000000000000  0.0000000000000000  0.0000000000000000
  0.0000000000000000  0.0000000000000000  1.5500000000000000
  ```

The section that contains the matrix,

<table>
<tr>
    <td>10.0000000000000000</td>
    <td>0.0000000000000000</td>
    <td>0.0000000000000000</td>
</tr>
<tr>
    <td>0.0000000000000000</td>
    <td>10.0000000000000000</td>
    <td>0.0000000000000000</td>
</tr>
<tr>
    <td>0.0000000000000000</td>
    <td>0.0000000000000000</td>
    <td>10.0000000000000000</td>
</tr>
    
</table>

denotes the geometry of the lattice vectors. In this case, the lattice vectors are the same as the cartesian basis vectors and each lattice vector extends 10 angstroms in their respective directions.

The lines that follow indicate what atoms are in the unit cell, and where they're located. In the case above,
```
N
    2
```
denotes that there are two nitrogen atoms. The lines that follow show where the atoms are. 
```
Cartesian
  0.0000000000000000  0.0000000000000000  0.0000000000000000
  0.0000000000000000  0.0000000000000000  1.5500000000000000
```
The first line says that the coordinates are given in the *Cartesian* basis. This line can also say *Direct*, which signifies that the positions are fractional positions in the lattice vector basis. [You can find more info about this distinction here](https://vasp.at/wiki/index.php/POSCAR). The second line shows that the first N atom is at the origin, while the third line shows the second N atom is on the z axis, 1.55 angstoms from the origin.

This POSCAR is ready to be used to build an in file. Start by making a directory in your ```tutorial``` directory in ```scratch``` called ```N2```. ```cd``` into the directory and copy over the POSCAR from the ```files/N2/``` directory in the tutorial files.
<img src="../Pictures/JDFTx/N2_POSCAR.png" alt="N2 POSCAR">

We need to convert the POSCAR into a file format that JDFTx can understand. To do this, run the script
```python
pos2jdftx.py
```

<img src="../Pictures/JDFTx/pos2jdftx.png" alt="pos2jdftx">

We see that the script created two new files, ```init.ionpos``` and ```init.lattice```. Examine these files with the ```cat``` command and you'll find how similar they look to different parts of the POSCAR. You may notice one key difference in the numbers, which arises from the fact that __JDFTx uses Bohrs__ while the POSCAR is written in Angstroms. This conversion must be done or the calculations will likely fail.

Now we can begin to build our in file. If you want more practice with this, see a similar tutorial in [the JDFTx documentation](http://jdftx.org/CoulombTrunc.html). To begin, ```touch``` a new file called ```N2.in```, and open it in ```nano```.
<img src="../Pictures/JDFTx/Touch_N2.png" alt="Touch N2">

First, we need to include the structure data. To do this, type
```
include init.ionpos
include init.lattice
```
This is C syntax, which is what JDFTx is written in

We need to include what are called _pseudopotentials_, which approximate the core electrons so that the DFT calculation can be simplified by only running calculations on the outer-shell electrons. Our group uses the GBRV1.5 psudeopotentials, which are installed at this path: ```/projects/musgravc/apps/jdftx-1.6.0/build/pseudopotentials/```. We tell JDFTx which pseudopotentials to use with the following command:
```
ion-species /projects/musgravc/apps/jdftx-1.6.0/build/pseudopotentials/GBRV_v1.5/$ID_pbe_v1.uspp
ion-species /projects/musgravc/apps/jdftx-1.6.0/build/pseudopotentials/GBRV_v1.5/$ID_pbe_v1.uspp
ion-species /projects/musgravc/apps/jdftx-1.6.0/build/pseudopotentials/GBRV_v1.5/$ID_pbe_v1.2.uspp
ion-species /projects/musgravc/apps/jdftx-1.6.0/build/pseudopotentials/GBRV_v1.5/$ID_pbe_v1.4.uspp
ion-species /projects/musgravc/apps/jdftx-1.6.0/build/pseudopotentials/GBRV_v1.5/$ID_pbe_v1.5.uspp

```

Psuedopotentials are quite a complex topic with many researchers dedicating their careers to developing them. Charles could give a good lecture about them if you want a stronger understanding of the theory.

We must also specify an electronic cutoff, which tells the software how many terms we want in our orbital expansion functions. Higher cutoff energies lead to higher computational cost but better orbitals (hopefully). In our case, we'll specify the cutoff energy with the tag,
```
elec-cutoff 20 100
```
If you read other papers, you'll see that their cutoff energies are well into the hundreds normally. This is because those papers are likely using _electronvolts_ and __JDFTx uses Hartree__. Again, you must remember to make this conversion or the calculation will never converge. The first number specifies the plane wave cutoff and the second number specifies the charge density cutoff. You can read more about this tag [here.](https://https://jdftx.org/CommandElecCutoff.html)

Next, we are going to truncate the coulomb interaction. This step is not necessary, but it will speed up the calculation. Check out the graphic below for a visual depiction of what coulomb truncation does
<img src="../Pictures/JDFTx/Coulomb_Truncation.png" alt="Coulomb Truncation">

Since this is a molecule, according to the [documentation](http://jdftx.org/CommandCoulombInteraction.html), we want to set the coulomb interaction to Isolated. We'll also set the coulomb-truncation-embed to 0 0 1.4645. This coordinate represents the middle of the molecule. If the coulomb-truncation-embed tag isn't the center of the molecule/surface, the calculation will fail. Add these lines to your ```N2.in``` file.
```
coulomb-interaction Isolated
coulomb-truncation-embed 0 0 1.4645
```
The coulomn-truncation-embed number is the center of the molecule. If our N2 molecule were centered at the origin, we would specify 0 0 0 as the argument of that tag

JDFTx has a plethora of data that it can output after a calculation, so we have to tell it what data we need. To sepcify the data JDFTx outputs, use the tag ```dump```. We want to dump the data needed to start a new calculation, some statistics about the final energy, and the final electron density. To get that output data, type the following lines in the ```N2.in```

```
dump-name N2.$VAR
dump End State
dump End Dtot
dump End Ecomponents
```


The last step in setting up the in file is to set the geometry optimization constraints. So far, all of the tags we've input have had to do with the electronic structure calculation. The following tags are used to take the forces generated from the electronic structure and use them to change the nuclear positions. We'll specify both force and energy convergence criteria. The energy convergence criteria is ```energyDiffThreshold 1e-6```, which means that the calculation will be converged if two consecutive calculations are within 1e-6 Hartree of each other. The force crtieria is ```knormThreshold 1e-4```, which means that the calculation will be converged if the [norm](https://mathworld.wolfram.com/VectorNorm.html) of the force vector is below 1e-4 Hartree/Bohr. We'll also add a maximum number of nuclear steps before the calculation finishes. We do this by setting the ```nIterations``` tag to the number of maximum steps. All of these tags are sub-tags to the ionic-minimize tag.

Add these lines to the ```N2.in``` file:
```
ionic-minimize \
    nIterations 10 \
    energyDiffThreshold 1e-6 \
    knormThreshold 1e-4
```

The file is complete! It should look like this:
<img src="../Pictures/JDFTx/final_in.png" alt='Final in'>

I included the ```init.DOS``` file in mine, which is not necessary.

We now need to build the submission script that slurm will use to run our job. Start by making a new file called ```submit.sh```

In the submission script, type:
```bash
#!/bin/bash
#SBATCH -J N2
#SBATCH --time=00:30:00
#SBATCH --tasks 1
#SBATCH --nodes 1
#SBATCH --ntasks-per-node 1
#SBATCH --account=ucb-general
#SBATCH --partition amilan
#SBATCH --qos=normal
```
Look up the SBATCH tags above to try and determine what each of them do. [This](https://slurm.schedmd.com/sbatch.html) should help.

To run the script, we need to use a program called MPI. MPI is a parallelization service that allows you to easily parallelize jobs across multiple cpus. In this case, we are just going to use 1 cpu without any parallelization. To use MPI, we first need to load the module and export a necessary environment variable.
```bash
export I_MPI_FABRICS=shm
module load intel impi
```

The script is run with the following two lines:
```bash
mpirun -np 1 /home/nisi6161/jdftx_alpine/build/jdftx -i N2.in -o out
exit 0
```
The ```-np 1``` tag specifies one _process_, meaning only one cpu core will be used to run the calculation. The -i line specifies the input file and the -o tag specifies the output.

Your ```submit.sh``` script should look like this
<img src="../Pictures/JDFTx/submit.png" alt="submit">

Your directory should look like this before running the calculation:
<img src="../Pictures/JDFTx/Ready_Directory.png" alt="ready directory">
It's okay if you don't have the init.DOS file as it won't affect the calculation

Once you're ready to submit the calculation, submit the calculation with ```sbatch```
```bin
sbatch submit.sh
```

You can check the calculation progress using ```qme```. After it has started running, a new ```out``` file should appear. JDFTx will write to this file as it continues the calculation, so you can use the file to check the calculation progress.

## Calculation Analysis

We can check to see if the calculation is converged by looking at the ```out``` file. To look at the last 40 lines of the file, type:
```bash
tail -n 40 out
```

The out file will look like this if it completed successfully
<img src="../Pictures/JDFTx/Tail_Out.png" alt="Tail Out">

If your out file looks like this, the calculation is converged and you can keep going in the tutorial. If it printed some sort of error message, consult a senior group member or the documentation for troubleshooting help.

Once the calculation converges, your calculation directory should look like this:
<img src="../Pictures/JDFTx/Calculation_Directory.png" alt="Calculation Directory">

There are a few files we haven't seen before. We'll go through each of them starting with the ```N2.Ecomponents``` file

```cat``` the N2.Ecomponents file to get some stats about the energetics. It should look like this:

<img src="../Pictures/JDFTx/N2_Ecomponents.png" alt="N2 Ecomponents">

You can see that JDFTx gave the total energy as -19.96H and listed all of the components that go into that number. This Etot is the *electronic energy*, and should not be thought of as an enthalpy or free energy, because it is missing thermochemical corrections such as entropy and heat capacity. The Etot value is not directly comparable to experiment.

### Structure

Next, let's look at the ```N2.ionpos``` file. This file is the converged structure, which we can compare to the starting file in VESTA. To download VESTA, go to [this website](https://jp-minerals.org/vesta/en/).

We need to convert the .ionpos structure to a file type that VESTA can read. In this case, we are going to convert it into a ```CONTCAR```, which is essentially just a POSCAR that has been converged in DFT. To do this, run ```j2pos.py``` in the N2/ directory

#### note
If the script throws an error and says you need something called 'octave,' you need to switch to an anaconda environment with octave installed in it. I have set up a jdft environment on Alpine. In order to access it, copy the ```.condarc``` file into your home directory on Alpine. The file can be found in the files/ directory of the tutorial. Once it's copied, source your .bashrc or restart your Putty session. Then type ```conda activate jdft``` to load the anaconda environment. Now you should be able to run the j2pos.py script

After running the script successfully, you should notice two new files

<img src="../Pictures/JDFTx/j2pos.png" alt="j2pos">

The N2.xsf file and the CONTCAR files are both structure files that are readable by VESTA, but the CONTCAR is more convenient to use within our group.

To visualize the CONTCAR in VESTA, you first need to copy it to your computer. To do so, drag it to your computer in WinSCP. I have a special directory I use for copying over and visualizing structure files on my Desktop. Once it's copied over, you can drag it directly from WinSCP (on the local computer side) to VESTA.

<img src="../Pictures/JDFTx/Vesta_Drag.png" alt="Vesta Drag">


Now that it's in VESTA, let's look at the bond length. To do so, use the measuring tool in VESTA and click on both of the nitrogen atoms.

<img src="../Pictures/JDFTx/N2_Vesta.png" alt="N2 Vesta">

We see that VESTA printed the bond length to the text window below. VESTA says our converged bond length is 1.108A, which is smaller than the starting bond length of 1.55A. Our ionic minimize algorithm pushed the nitogen atoms together to the lowest energy configuration! You'll find that the converged bond length is much closer to the correct answer than our starting guess.

## Summary

You can now run molecular calculations in JDFTx! You can continue on to the Pt_111 tutorial